In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import random
from time import time


from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util



import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as models
from torchvision import datasets, models, transforms
import os
import time

In [8]:
resnet = models.resnet50(pretrained=True)
new_classifier = nn.Sequential(*list(resnet.fc.children())[:-1])
resnet = resnet.fc.in_features

In [ ]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
ds_trans = transforms.Compose([transforms.Scale(224),
                               transforms.CenterCrop(224),
                               transforms.ToTensor(),
                               normalize])




data = []
labels = []

t0 = time()

image_feature = np.empty(shape=[0,4096])
for filename in sorted(os.listdir('training_set/')):
    input_tensor = []
    img = cv2.imread('training_set/'+filename)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    tensor_img = ds_trans(img)


    features = vgg16(torch.stack([left_up,right_up,center,left_down,right_down]))
    output_features = torch.sum(features,dim = 0)   #return size [1,4096]
    output_features = output_features*1.0 / 5
    image_feature = np.vstack((image_feature,output_features.detach().numpy()))

print(image_feature.shape)
scipy.io.savemat('UCF101_release/vgg16_relu6/'+filename,{'Feature':image_feature},do_compression = True)
print("Writing to "+'UCF101_release/vgg16_relu6/'+filename+'.mat')





In [ ]:
use_gpu = torch.cuda.is_available()
inputs, labels = next(iter(train_dl))
if use_gpu:
    resnet = resnet.cuda()
    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())   
else:
    inputs, labels = Variable(inputs), Variable(labels)
outputs = resnet(inputs)
outputs.size()

In [ ]:
# freeze all model parameters
for param in resnet.parameters():
    param.requires_grad = False

# new final layer with 16 classes
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, 16)
if use_gpu:
    resnet = resnet.cuda()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet.fc.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [4]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()
    
    for epoch in range(1,num_epochs + 1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)
        phase = 'train'
        model.train()  # Set model to training mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to('cpu')
            labels = labels.to('cpu')

            # zero the parameter gradients
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = (running_corrects.double() / len(dataloaders[phase].dataset))*100

        print('{} Loss: {:.4f} Acc: {:.4f}%'.format(phase, epoch_loss, epoch_acc))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return model

In [5]:
def test_model(model , dataloaders):
    model.eval()
    since = time.time()
    total = 0
    correct = 0
    phase = 'test'
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to('cpu')
        labels = labels.to('cpu')
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    epoch_acc = ((correct*1.0) / len(dataloaders['test'].dataset)) *100

    print('{} Acc: {:.4f}%'.format(phase, epoch_acc))

    time_elapsed = time.time() - since
    print('Testing complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()
num_epochs = 30
# Train and evaluate
model_ft  = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

In [ ]:
test_model(model_ft, dataloaders_dict)

In [5]:
model = nn.Linear(20, 5) # predict logits for 5 classes
x = torch.randn(1, 20)
y = torch.tensor([[1., 0., 1., 0., 0.]])

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

for epoch in range(20):
    optimizer.zero_grad()
    output = model(x)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    print('Loss: {:.3f}'.format(loss.item()))

Loss: 0.843
Loss: 0.675
Loss: 0.551
Loss: 0.457
Loss: 0.387
Loss: 0.333
Loss: 0.291
Loss: 0.257
Loss: 0.230
Loss: 0.207
Loss: 0.189
Loss: 0.173
Loss: 0.159
Loss: 0.148
Loss: 0.138
Loss: 0.129
Loss: 0.121
Loss: 0.114
Loss: 0.108
Loss: 0.102
